In [1]:
import os

In [2]:
# check working directory
%pwd

'f:\\Project\\Text_Summarization\\research'

In [3]:
# changing work dir
os.chdir('../')
%pwd

'f:\\Project\\Text_Summarization'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataInjestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [6]:
class ConfigManager:
    def __init__(self,
                 config_path = CONFIG_FILE_PATH,
                 params_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)

        create_directories([self.config.artifacts_root])
    
    def get_data_injestion_config(self)->DataInjestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_injestion_config = DataInjestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )

        return data_injestion_config

In [7]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [8]:
class DataInjestion:
    def __init__(self, config:DataInjestionConfig):
        self.config = config

    def download_files(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f'{filename} downloaded! with info: {headers}')
        else:
            logger.info(f'File already exists of size: {get_size(Path(self.config.local_data_file))}')

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok= True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [9]:
try:
    config_manager = ConfigManager()
    data_injestion_config = config_manager.get_data_injestion_config()
    data_injestion = DataInjestion(data_injestion_config)
    data_injestion.download_files()
    data_injestion.extract_zip_file()
except Exception as e:
    raise e

[2024-01-04 21:31:45,839 : INFO : common : yaml file : config\config.yml loaded succefully]
[2024-01-04 21:31:45,843 : INFO : common : yaml file : params.yml loaded succefully]
[2024-01-04 21:31:45,847 : INFO : common : Created Director at: artifacts]
[2024-01-04 21:31:45,847 : INFO : common : Created Director at: artifacts/data_ingestion]
[2024-01-04 21:31:45,847 : INFO : 1097642537 : File already exists of size: ~ 7718 KB]
